# Ego networks 

In this notebook, we will build the ego networks using the two dataframes of clean data obtained in the past notebook _clean.ipynb_. In order to achieve this: 

* We will use the library _networkx_ to transform the dataframe of relationships between the alteri into the different ego networks. 
* We will extract some measures of the structure of this ego networks and create with them a dataframe, as our goal is to use them as predictors of the nationality.

## Load the .csv files 

We import the different libraries, the usual numpy, pandas, matplotlib and networkx. The methods library that can be seen contains the algorithms to calculate the Dunbar estructure of an individual given an ego network. We will also incorporate this measure in our analysis in order to provide a more complete view. Then we load our .csv files and delete the unformatted columns.

In [1]:
###Import libraries
from numpy import nan
import pandas as pd 
import networkx as nx
from methods import *
import numpy as np
import matplotlib 
from matplotlib import pyplot as plt
import warnings
#warnings.filterwarnings('ignore')

###Load .csv files
contactos=pd.read_csv(r'/home/juan/Python/Acculturation/Contactos.csv',low_memory=False)
df=pd.read_csv(r'/home/juan/Python/Acculturation/all_data_clean.csv',low_memory=False)

###Delete useless columns
del contactos['Unnamed: 0']
del df['Unnamed: 0']


## Find out the total number of subjects

In order to begin the calculations, we calculate the total number of subjets. We also transform the datatype of the columns that are going to play a role in the dessign of the ego networks. 

In [2]:
###Change datatypes
contactos['Alter']=contactos['Alter'].astype(int)
contactos['Alter2']=contactos['Alter2'].astype(int)

#Find out the total number of subjects 
sujetos = len(contactos['sub/num'].unique())
print("The total number of subjects is {}".format(sujetos))

The total number of subjects is 473


## Creating the ego networks and measuring their properties 

At this point, we build the ego networks. Coming from the contacts dataframe, we build a _networkx_ graph and we go through all the subjects, one by one, introducing the ties between alteris as links between nodes. And the intensity of their relationships determines the weight associated with the link. 
<br> <br>
Once we have these graphs, we compute different properties: _Average degree, betweenness, closeness, clustering, load centrality, size of the largest component, number of components.._ Apart from these structural measures, we also compute the average intensity with people from their origin country and people from their residence country and their presence (the total number in the ego network). We store all these values in different lists. 

In [3]:
def include_ego(grafica):
    edges_ego = []
    for node in grafica.nodes():
        edges_ego.append((0,node,{"Value":1}))
    grafica.add_edges_from(edges_ego)
    return grafica

In [4]:
###We create the list in order to store the ego networks and their different properties. 
graficas=[0]*sujetos
avdeg=[0]*sujetos
betw=[0]*sujetos
closs=[0]*sujetos
assort=[0]*sujetos
clustering=[0]*sujetos
load=[0]*sujetos
size=[0]*sujetos
comp=[0]*sujetos
ori=[0]*sujetos
num=[0]*sujetos
res=[0]*sujetos
#closnac=[0]*sujetos
#closnonac=[0]*sujetos
#numnac=[0]*sujetos
#mu=[0]*sujetos
#numnonac=[0]*sujetos
#vect=[[0]*5 for i in range(sujetos)]

###We compute these ego networks and their attributes

for k,j in enumerate(contactos["sub/num"].unique()):
    graficas[k]=nx.Graph()
    ###We select the data that corresponds to each subject
    data=contactos[contactos['sub/num']==j]
    if (len(data)>0):
        ###We compute the different number of subjects and their intensity. 
        ###This will be used to calculate the Dunbar parameters. 
        #datamu=df['Clos'][df['sub/num']==j].value_counts()
        #for m in range(0,len(datamu)):
        #    vect[k][datamu.index[m]-1]= datamu[datamu.index[m]]
        ###Building the ego networks
        edges=list(zip(data['Alter'],data['Alter2'],data['Value']))
        graficas[k].add_weighted_edges_from(edges,'Value')
        #graficas[k] = include_ego(graficas[k])
        ###Measuring the networks
        degrees=[val for (node, val) in graficas[k].degree()]
        avdeg[k]=sum(degrees)/len(degrees)
        betw[k]=sum(nx.betweenness_centrality(graficas[k],weight='Value').values())/len(nx.betweenness_centrality(graficas[k]).values())
        closs[k]=sum(nx.closeness_centrality(graficas[k]).values())/len(nx.closeness_centrality(graficas[k]).values())
        load[k]=sum(nx.load_centrality(graficas[k],weight='Value').values())/len(nx.load_centrality(graficas[k]).values())
        assort[k]=nx.degree_assortativity_coefficient(graficas[k],weight='Value')
        size[k]= len(max(nx.connected_components(graficas[k]), key=len))/len(graficas[k].nodes())
        clustering[k]=nx.average_clustering(graficas[k],weight='Value')
        
        comp[k]=nx.number_connected_components(graficas[k])
        ego_origin = df['sub/origin'][df['sub/num'] == j].unique()[0]
        ego_residence = df['sub/residence'][df['sub/num'] == j].unique()[0]
        #closnac[k]=df['Clos'][(df['sub/num'] == j) & (df['alter/origin'] == ego_origin)].mean() 
        #numnac[k]=df['Clos'][(df['sub/num'] == j) & (df['alter/origin'] == ego_origin)].count()
        #closnonac[k]=df['Clos'][(df['sub/num'] == j) & (df['alter/origin'] == ego_residence)].mean()
        #numnonac[k]=df['Clos'][(df['sub/num'] == j) & (df['alter/origin'] == ego_residence)].count()
        ori[k]=ego_origin
        num[k]=j
        res[k]=ego_residence
    else: print(j)




/home/juan/.local/lib/python3.8/site-packages/networkx/algorithms/assortativity/correlation.py:282: RuntimeWarning: invalid value encountered in double_scalars
  return (xy * (M - ab)).sum() / np.sqrt(vara * varb)


## Create the ego networks dataframe 

Coming from the previous lists, we create the networks dataframe, that contains the ego networks and their attributes. At this point, we also compute the Dunbar's parameter $\mu$ and add it to our dataframe, with its confidence interval. 

In [5]:
###Creation of the dataframe
#redes=pd.DataFrame(data={'Subject origin':ori,'Subject residence':res,'Subject ID':num,'Vect':vect,'Graphs':graficas,
#                         'Average degree':avdeg,'Betweenness':betw,'Closeness':closs,'Load centrality':load,
#                         'Assortativity':assort,'Clustering':clustering,
#                         'Number components':comp,'Size largest component':size,
#                         'Closeness residence':closnonac,'Closeness origin':closnac,'Number residence':numnonac,
#                         'Number origin':numnac})
redes=pd.DataFrame(data={'Subject origin':ori,'Subject residence':res,'Subject ID':num,'Graphs':graficas,
                         'Average degree':avdeg,'Betweenness':betw,'Closeness':closs,'Load centrality':load,
                         'Assortativity':assort,'Clustering':clustering,
                         'Number components':comp,'Size largest component':size})

df.isnull().sum()
redes.fillna(0,inplace=True)
###Calculating the Dunbar's parameters 
#redes['Fitted']=redes['Vect'].apply(lambda x:Individual(x).fit_model())
#redes[['Mu','Mu-','Mu+']] = pd.DataFrame(redes.Fitted.values.tolist(), index= redes.index)

## Some changes in the dataframe 

At this point, we make some changes in the ego networks dataframe. We introduce a function that substitutes the confidence interval of the $\mu$ by an string that determines the regime. Then we delete the columns associated with the confidence interval, fill the _NaN_ and rename some columns.

In [6]:
###Creating function to make clearer the regime
def rex(x,y):
    if ((x < 0) & (y < 0)): z='Inverted'
    elif ((x > 0) & (y > 0)): z='Standard'
    else : z='Unclear'
    return z

###Introducing the former function in our dataframe
#regime=[0]*sujetos
#for i in range(len(redes)):
#    regime[i]=rex(redes['Mu-'][i],redes['Mu+'][i])
#redes['Regime']=regime
#del redes['Fitted']
#del redes['Mu-']
#del redes['Mu+']

###Filling NaNs
#redes.fillna(0,inplace=True)

###Renaming columns and getting rid of the networkx graph
#redes=redes[['Subject ID','Subject origin','Subject residence','Mu','Regime','Average degree','Betweenness',
#             'Closeness','Load centrality','Assortativity','Clustering',
#             'Number components','Size largest component',
#             'Closeness residence','Number residence','Closeness origin','Number origin','Graphs']]
redes=redes[['Subject ID','Subject origin','Subject residence','Average degree','Betweenness',
             'Closeness','Load centrality','Assortativity','Clustering',
             'Number components','Size largest component','Graphs']]

del redes['Graphs']

## More changes in the networks dataframe and merging 

Now we are going to merge the network measures dataframe with the original one _df_, that contains information about egos and alteris. We want to preserve the egos attributes, so we delete all the information about the alteris. We will also introduce the variable _FMIG2_ that measures the number of years these migrants have spent in their residence countries. Once we have made all these changes, some typos need to be corrected, and that is what we do at the end of this cell. 

In [7]:
###Deleting the alteri column in df and merging it with the networks dataframe
df.drop(["alter/origin","alter/residence","alter/num"],axis=1,inplace=True)
df.drop([col for col in df.columns if col[0] == "A"],axis=1,inplace=True)
df.rename(columns={'sub/num':'Subject ID','sub/origin':'Subject origin','sub/residence':'Subject residence'},inplace=True)
df = df.drop_duplicates(['Subject origin','Subject ID'])

redes2= pd.merge(redes, df,how='left',on=['Subject ID','Subject origin','Subject residence'])
redes2.drop("Subject ID",axis=1,inplace=True)

###Calculating FMIG2
import datetime
FMIG2=[0]*len(redes2)
for i in range(len(redes2)):
    if (redes2['FMIG'].iloc[i]<0.5): 
        FMIG2[i]=0
    elif (redes2['FMIG'].iloc[i]>1500):
        FMIG2[i] = 2005 - redes2['FMIG'].iloc[i]
redes2['FMIG2']=FMIG2
redes2.drop("FMIG",axis=1,inplace=True)

###Renaming columns and changing some typos
redes2.rename(columns={'Subject origin':'Subject_origin','Subject residence':'Subject_residence'},inplace=True)
redes2.columns = redes2.columns.str.replace(' ', '_')
redes2.dropna(axis=1,inplace=True)


## Save the final results and display a sample

In [8]:
redes2.to_csv("Redes_2.csv")

In [9]:
nx.__version__

'2.6.3'

In [10]:
redes2.sample(10)

,Subject_origin,Subject_residence,Average_degree,Betweenness,Closeness,Load_centrality,Assortativity,Clustering,Number_components,Size_largest_component,...,TIEDEN,TIEDC,TIECC,TIEBC,TIECND,TIECCZ,DSET,sub/language,alter_language,FMIG2
136,pu,usa,44.000000,0.005251,1.000000,0.004416,-0.022727,0.443452,1,1.000000,...,0.18990,78.66287,2332.70721,7.66692,11.0,0.90,rc1f1pre,en,en,0.0
440,se,sp,7.955556,0.027030,0.413990,0.027028,0.177088,0.603074,2,0.955556,...,0.12929,83.24420,39.04754,30.20888,4.0,0.81,l3bs,es,es,2.0
216,ar,sp,9.644444,0.029058,0.454121,0.029058,0.001705,0.643503,1,1.000000,...,0.21919,81.71247,29.39773,28.89688,8.0,0.69,l5ca,es,es,-1.0
61,do,usa,15.466667,0.015520,0.544001,0.015518,-0.008601,0.592477,2,0.955556,...,0.17677,85.38206,8419.10629,34.55963,8.0,0.79,rc2ds,es,es,0.0
426,se,sp,22.488889,0.011443,0.680561,0.011435,-0.138651,0.658373,1,1.000000,...,0.45051,55.37099,63.79310,8.60800,15.0,0.47,l2bs,es,es,11.0
104,do,usa,44.000000,0.002744,1.000000,0.002607,-0.022727,0.390453,1,1.000000,...,0.11212,84.07258,5171.17767,1.72071,7.0,0.91,rc2de,en,en,0.0
217,ar,sp,12.933333,0.016420,0.594448,0.016420,-0.177886,0.771905,1,1.000000,...,0.29394,73.89007,83.91839,45.84123,14.0,0.73,l5da,es,es,16.0
265,ar,sp,9.555556,0.018205,0.569890,0.018205,-0.374783,0.773919,1,1.000000,...,0.21717,81.92389,89.00012,56.72185,8.0,0.84,l2aa,es,es,4.0
420,se,sp,22.977778,0.011113,0.680042,0.011113,-0.110167,0.529352,1,1.000000,...,0.50000,52.32558,39.81595,2.83102,22.0,0.34,l1s,es,es,0.0
70,do,usa,44.000000,0.003270,1.000000,0.002161,-0.022727,0.391767,1,1.000000,...,0.09293,94.19280,3398.46306,7.04420,4.0,0.87,rc2f1de,en,en,0.0
